In [1]:
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.cross_validation import train_test_split
import tflearn
import joblib

In [2]:
data = np.load('data/combined_data.npy')

In [3]:
X = data[:,:-3]
y = data[:,-1].reshape(-1,1)

In [4]:
print X.shape, y.shape

(80000, 10) (80000, 1)


In [5]:
X_tr, X_t, y_tr, y_t = train_test_split(X, y, test_size=0.3)

In [6]:
l0 = tflearn.input_data(shape=[None,10])
l1 = tflearn.fully_connected(l0, 100, activation='relu')
l2 = tflearn.fully_connected(l1, 100, activation='relu')
l3 = tflearn.fully_connected(l2, 50, activation='relu')
lf = tflearn.fully_connected(l3, 1)
net = tflearn.regression(lf, loss="mean_square", restore=False)

In [7]:
model = tflearn.DNN(net)

In [12]:
model.fit(X_tr, y_tr, n_epoch=60)

Training Step: 79222  | total loss: 856449920.00000
| Adam | epoch: 060 | loss: 856449920.00000 -- iter: 56000/56000
Training Step: 79222  | total loss: 856449920.00000
| Adam | epoch: 060 | loss: 856449920.00000 -- iter: 56000/56000
--


In [13]:
y_pred = model.predict(X_tr)

In [14]:
mae = mean_absolute_error(y_pred, y_tr)
r2 = r2_score(y_pred, y_tr)
print mae, r2

20059.1767432 -6.00177651517


In [19]:
print np.mean(y_tr)

33915.7766429


In [16]:
# saving model
data = {
    'l1W':model.get_weights(l1.W),
    'l1b':model.get_weights(l1.b),
    'l2W':model.get_weights(l2.W),
    'l2b':model.get_weights(l2.b),
    'l3W':model.get_weights(l3.W),
    'l3b':model.get_weights(l3.b),
    'lfW':model.get_weights(lf.W),
    'lfb':model.get_weights(lf.b),
}
joblib.dump(data, 'models/model.tfl', compress=3)

['models/model.tfl']

In [ ]:
# loading model
data_loaded = joblib.load('models/model.tfl')
model_loaded = tflearn.DNN(net)
model_loaded.set_weights(l1.W, data_loaded['l1W'])
model_loaded.set_weights(l1.b, data_loaded['l1b'])
model_loaded.set_weights(l2.W, data_loaded['l2W'])
model_loaded.set_weights(l2.b, data_loaded['l2b'])
y_pred = model.predict(X_t)

In [ ]:
y_pred2 = model_loaded.predict(X_t)
mae = mean_absolute_error(y_pred2, y_t)
r2 = r2_score(y_pred2, y_t)
print mae, r2

In [7]:
model2 = tflearn.DNN(net, tensorboard_verbose=3)

In [8]:
model2.fit(X_tr, y_tr, n_epoch=1)

Training Step: 875  | total loss: 1085846144.00000
| Adam | epoch: 001 | loss: 1085846144.00000 -- iter: 56000/56000
Training Step: 875  | total loss: 1085846144.00000
| Adam | epoch: 001 | loss: 1085846144.00000 -- iter: 56000/56000
--
